### Get CSV Files

In [1]:
import os

current_directory = os.getcwd()
csv_files = [file for file in os.listdir(current_directory) if file.endswith('.csv')]
csv_files

['2014_LoL_esports_match_data_from_OraclesElixir.csv',
 '2015_LoL_esports_match_data_from_OraclesElixir.csv',
 '2016_LoL_esports_match_data_from_OraclesElixir.csv',
 '2017_LoL_esports_match_data_from_OraclesElixir.csv',
 '2018_LoL_esports_match_data_from_OraclesElixir.csv',
 '2019_LoL_esports_match_data_from_OraclesElixir.csv',
 '2020_LoL_esports_match_data_from_OraclesElixir.csv',
 '2021_LoL_esports_match_data_from_OraclesElixir.csv',
 '2022_LoL_esports_match_data_from_OraclesElixir.csv',
 '2023_LoL_esports_match_data_from_OraclesElixir.csv',
 '2024_LoL_esports_match_data_from_OraclesElixir.csv',
 '2025_LoL_esports_match_data_from_OraclesElixir.csv']

### Turn to Dataframes and Preprocess


In [2]:
import numpy as np
import pandas as pd
import DataClasses as dc

In [3]:
def get_dataframe(csv_file: str) -> pd.DataFrame:
    df = pd.read_csv(csv_file)
    df = df[df['datacompleteness'] == 'complete']

    relevant_features = ['gameid', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'side', 'position', 'playername', 'teamname', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'gamelength', 'result', 'kills', 'deaths', 
                         'assists', 'firstblood', 'firstdragon', 'dragons', 'infernals', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)', 'elders', 'firstherald', 'heralds', 'void_grubs', 'firstbaron', 
                         'barons', 'firsttower', 'towers', 'turretplates', 'inhibitors', 'damagetochampions', 'wardsplaced', 'wardskilled', 'controlwardsbought', 'visionscore', 'totalgold', 'goldspent', 'total cs', 'goldat15', 'killsat15', 'assistsat15', 'deathsat15']
    
    df = df[relevant_features]
    return df

### Going Through Data

In [4]:
import math

def get_tournaments(df: pd.DataFrame):
    df_tour = df[['league', 'year', 'split']]
    df_tour = df_tour.drop_duplicates().reset_index(drop=True)

    tournaments = []

    for row in df_tour.itertuples():
        tournament = dc.Tournament(row[1], row[2], row[3])
        tournament.fix_split()
        tournaments.append(tournament)

    return tournaments

def get_teams(df: pd.DataFrame):
    df_teams = df[['teamname', 'year']]
    df_teams = df_teams.drop_duplicates().reset_index(drop=True)

    teams = []

    for row in df_teams.itertuples():
        team = dc.Team(row[1], row[2])
        team.fix_name()
        teams.append(team)

    return teams

def get_players(df: pd.DataFrame):
    df_players = df[['position', 'playername', 'teamname', 'year']]
    df_players = df_players.drop_duplicates().reset_index(drop=True)

    players = []

    for row in df_players.itertuples():
        if row[1] == 'team':
            continue

        team = dc.Team(row[3], row[4])
        team.fix_name()

        player = dc.Player(row[2], team)
        player.fix_name()
        players.append(player)

    return players

def get_objectives(df: pd.DataFrame):
    df_objectives = df[df['position'] == 'team'][['side', 'firstblood', 'firsttower', 'towers', 'turretplates', 'void_grubs', 'heralds', 'barons', 'infernals', 'mountains', 'clouds', 'oceans', 'hextechs', 'chemtechs', 'elders']]
    
    blue_objectives = None
    red_objectives = None

    for row in df_objectives.itertuples():
        objectives = dc.Objectives(row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], False, 0, 0)
        objectives.fix_objectives()
        if row[1] == 'Blue':
            blue_objectives = objectives
        else:
            red_objectives = objectives

    return (blue_objectives, red_objectives)


def get_pick_bans(df: pd.DataFrame):
    df_pick_bans = df[df['position'] == 'team'][['side', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5']]
    
    blue_pick_bans = None 
    red_pick_bans = None 

    for row in df_pick_bans.itertuples():
        pick_bans = dc.PickBan(row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11])
        pick_bans.check_existing_champions()
        
        if row[1] == 'Blue':
            blue_pick_bans = pick_bans
        else:
            red_pick_bans = pick_bans

    return (blue_pick_bans, red_pick_bans)

def get_player_performances(df: pd.DataFrame):
    df_players = df[df['position'] != 'team'][['side', 'playername', 'position', 'champion', 'kills', 'deaths', 'assists', 
                     'damagetochampions', 'wardsplaced', 'wardskilled', 'controlwardsbought', 'visionscore', 
                     'totalgold', 'goldspent', 'total cs', 'killsat15','deathsat15', 'assistsat15','goldat15',
                     'teamname', 'year']]
    
    blue_player_performances = []
    red_player_performances = []

    for row in df_players.itertuples():
        team = dc.Team(row[20], row[21])
        team.fix_name()
        player = dc.Player(row[2], team)
        player.fix_name()
        player_performance = dc.PlayerPerformance(player, row[3], row[4], row[5], 
                                                  row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], 
                                                  row[14], row[15], row[16], row[17], row[18], row[19])
        player_performance.set_15_stats()
        player_performance.fix_stats()
        if row[1] == 'Blue':
            blue_player_performances.append(player_performance)
        else:
            red_player_performances.append(player_performance)

    return (blue_player_performances, red_player_performances)

def get_game(df: pd.DataFrame):
    df_game = df[df['position'] == 'team'][['game', 'side', 'position', 'teamname', 'year', 'gamelength', 'result']]

    blue_pb, red_pb = get_pick_bans(df)
    blue_obj, red_obj = get_objectives(df)
    blue_pp, red_pp = get_player_performances(df)

    blue_team = dc.Team(df_game[df_game['side'] == 'Blue']['teamname'].iloc[0], df_game[df_game['side'] == 'Blue']['year'].iloc[0])
    blue_team.fix_name()
    red_team = dc.Team(df_game[df_game['side'] == 'Red']['teamname'].iloc[0], df_game[df_game['side'] == 'Red']['year'].iloc[0])
    red_team.fix_name()

    duration = df_game['gamelength'].iloc[0]

    number = df_game['game'].iloc[0]

    blue_win = df_game[df_game['side'] == 'Blue']['result'].iloc[0]

    # game = dc.Game(number, blue_team, red_team, blue_win, blue_pb, red_pb, blue_obj, red_obj, blue_pp, red_pp, duration)
    # print(game)
    # print(df_game)
    # print(df_game[df_game['side'] == 'Blue']['year'].iloc[0])

    return dc.Game(number, blue_team, red_team, blue_win, blue_pb, red_pb, blue_obj, red_obj, blue_pp, red_pp, duration)

# Only insert tournament dfs 
def get_matches(df: pd.DataFrame):
    matches = []
    currentMatch = None

    df_matches = df[df['position'] == 'team'][['gameid', 'playoffs', 'date', 'game', 'patch', 'teamname', 'year']]

    # print(df_matches)

    for row in df_matches.itertuples():
        if row[4] == 1 or currentMatch is None:
            if currentMatch is not None:
                matches.append(currentMatch)
                # print(row[3].split()[0])
            
            currentMatch = dc.Match(date = row[3].split()[0], playoffs=row[2], patch=row[5])
            currentMatch.fix_patch()
            # print(type(row[5]), row[5], currentMatch.patch)
            teamOne = dc.Team(df_matches[(df_matches['gameid'] == row[1]) & (df_matches['game'] == row[4])]['teamname'].iloc[0], df_matches[(df_matches['gameid'] == row[1]) & (df_matches['game'] == row[4])]['year'].iloc[0])
            teamOne.fix_name()
            currentMatch.teamOne = teamOne
            teamTwo = dc.Team(df_matches[(df_matches['gameid'] == row[1]) & (df_matches['game'] == row[4])]['teamname'].iloc[1], df_matches[(df_matches['gameid'] == row[1]) & (df_matches['game'] == row[4])]['year'].iloc[0])
            teamTwo.fix_name()
            currentMatch.teamTwo = teamTwo 

        currentMatch.games.append(get_game(df[df['gameid'] == row[1]]))

    return matches

In [5]:
df_2024 = get_dataframe(csv_files[0])
# test = df_2024[df_2024['position'] == 'team'][['gameid', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'teamname']]
# test[test['league'] == 'LCK'].head(10)

tournament_df = df_2024[df_2024['league'] == 'EU CS']
tournament_df = tournament_df[tournament_df['year'] == 2014]
get_matches(tournament_df)

# tournament_df[tournament_df['position'] == 'team'][['gameid', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'teamname']].head(10)

# get_game(df_2024[df_2024['gameid'] == 'LOLTMNT03_183544'])
# # type(df_2024[df_2024['gameid'] == 'LOLTMNT03_183544'][df_2024['position'] == 'team'][['game', 'year', 'side', 'position', 'teamname', 'year', 'gamelength', 'result']][df_2024['side'] == 'Blue']['teamname'].iloc[0])
# type(df_2024[df_2024['side'] == 'Blue']['year'].iloc[0])

In [6]:
df_2024 = get_dataframe(csv_files[-1])
tournaments_2024 = get_tournaments(df_2024)
teams_2024 = get_teams(df_2024)
players_2024 = get_players(df_2024)

ppb_2024, ppr_2024 = get_player_performances(df_2024[df_2024['gameid'] == 'LOLTMNT03_183544'])

for tournament in tournaments_2024:
    print(tournament)
    tournament_df = df_2024[df_2024['league'] == tournament.league]
    tournament_df = tournament_df[tournament_df['year'] == tournament.year]
    tournament_df = tournament_df[tournament_df['split'] == tournament.split]

    print(tournament_df)

    tournament.matches = get_matches(tournament_df)
    break
# for team in teams_2024:
#     print(team)
# for player in players_2024:
#     print(player)
# for player in ppb_2024:
#     print(player)
# # ppb_2024

C:\Users\ktann\AppData\Local\Temp\ipykernel_22052\3123184168.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


LFL2 2025 - Winter
                 gameid league  year   split  playoffs                 date  \
0      LOLTMNT03_179647   LFL2  2025  Winter         0  2025-01-11 11:11:24   
1      LOLTMNT03_179647   LFL2  2025  Winter         0  2025-01-11 11:11:24   
2      LOLTMNT03_179647   LFL2  2025  Winter         0  2025-01-11 11:11:24   
3      LOLTMNT03_179647   LFL2  2025  Winter         0  2025-01-11 11:11:24   
4      LOLTMNT03_179647   LFL2  2025  Winter         0  2025-01-11 11:11:24   
...                 ...    ...   ...     ...       ...                  ...   
21199  LOLTMNT06_107351   LFL2  2025  Winter         1  2025-03-09 14:24:44   
21200  LOLTMNT06_107351   LFL2  2025  Winter         1  2025-03-09 14:24:44   
21201  LOLTMNT06_107351   LFL2  2025  Winter         1  2025-03-09 14:24:44   
21202  LOLTMNT06_107351   LFL2  2025  Winter         1  2025-03-09 14:24:44   
21203  LOLTMNT06_107351   LFL2  2025  Winter         1  2025-03-09 14:24:44   

       game  patch  side positio

### Extracting Needed Data Test

In [43]:
for csv in csv_files:
    df = get_dataframe(csv)
    tournaments = get_tournaments(df)
    teams = get_teams(df)
    players = get_players(df)

    for tournament in tournaments:
        tournament_df = df[df['league'] == tournament.league]
        tournament_df = tournament_df[tournament_df['year'] == tournament.year]
        tournament_df = tournament_df[tournament_df['split'] == tournament.split]

        tournament.matches = get_matches(tournament_df)

        print(tournament)

KeyboardInterrupt: 

In [ ]:
# for i in teams:
#     print(i)

### Connecting to DB 

In [25]:
# %pip install mysql-connector-python
# %pip install python-dotenv

In [6]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

database = mysql.connector.connect(
  host="localhost",
  user="root",
  password= os.getenv('mysql_password'),
  database = 'league_of_legends_esports'
)

if database.is_connected():
    print(f"Connected to db")
else:
    print(f"Failed to connect to db")

Connected to db


In [27]:
db_cursor = database.cursor()

db_cursor.execute("SHOW DATABASES")

for x in db_cursor:
  print(x) 

('information_schema',)
('league_of_legends_esports',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


### Disconnecting to DB

In [21]:
database.close()

### Adding to DB

#### SQL Statements

In [8]:
# Teams
insert_team = '''INSERT INTO teams (name, year) VALUES (%s, %s)'''
team_count = '''SELECT COUNT(*) FROM teams WHERE name = %s AND year = %s'''
team_id = '''SELECT id FROM teams WHERE name = %s AND year = %s'''

In [9]:
# Players
insert_player = '''INSERT INTO players (name, team) VALUES (%s, %s)'''
player_count = '''SELECT COUNT(*) FROM players WHERE name = %s AND team = %s'''
player_id = '''SELECT id FROM players WHERE name = %s AND team = %s'''

In [10]:
# Tournaments 
insert_tournament = '''INSERT INTO tournaments (league, year, split) VALUES (%s, %s, %s)'''
tournament_count = '''SELECT COUNT(*) FROM tournaments WHERE league = %s AND year = %s AND split = %s'''
tournament_id = '''SELECT id FROM tournaments WHERE league = %s AND year = %s AND split = %s'''

In [11]:
# Matches
insert_match = '''INSERT INTO matches (tournament_id, date, team_one, team_two, playoffs, patch) VALUES (%s, %s, %s, %s, %s, %s)'''
match_count = '''SELECT COUNT(*) FROM matches WHERE tournament_id = %s AND date = %s AND team_one = %s AND team_two = %s AND playoffs = %s AND patch = %s'''
match_id = '''SELECT id FROM matches WHERE tournament_id = %s AND date = %s AND team_one = %s AND team_two = %s AND playoffs = %s AND patch = %s'''

In [12]:
# Games
insert_game = '''INSERT INTO games (match_id, game_number, blue_team, red_team, blue_win, duration) VALUES (%s, %s, %s, %s, %s, %s)'''
game_count = '''SELECT COUNT(*) FROM games WHERE match_id = %s AND game_number = %s AND blue_team = %s AND red_team = %s AND blue_win = %s AND duration = %s'''
game_id = '''SELECT id FROM games WHERE match_id = %s AND game_number = %s AND blue_team = %s AND red_team = %s AND blue_win = %s AND duration = %s'''

In [13]:
# Objectives
insert_objective = '''
INSERT INTO 
objectives (game_id, side, first_blood, first_tower, towers, tower_plates, void_grubs, rift_heralds, baron_nashors, infernals, mountains, clouds, oceans, hextechs, chemtechs, elders, feats_of_strength, ruinous_atakhan, voracious_atakhan) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

objective_count = '''SELECT COUNT(*) FROM objectives WHERE game_id = %s AND side = %s AND first_blood = %s AND first_tower = %s AND towers = %s AND tower_plates = %s AND void_grubs = %s AND rift_heralds = %s AND baron_nashors = %s AND infernals = %s AND mountains = %s AND clouds = %s AND oceans = %s AND hextechs = %s AND chemtechs = %s AND elders = %s AND feats_of_strength = %s AND ruinous_atakhan = %s AND voracious_atakhan = %s'''

In [14]:
# Pick Bans
# insert_pick_ban = '''INSERT INTO pick_bans (game_id, side, ban1, ban2, ban3, ban4, ban5, pick1, pick2, pick3, pick4, pick5) 
# VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
# pick_ban_count = '''SELECT COUNT(*) FROM pick_bans WHERE game_id = %s AND side = %s'''

insert_pick_ban = '''Insert into pick_bans (game_id, side, pick_ban, order_number, champion) VALUES (%s, %s, %s, %s, %s)'''
pick_ban_count = '''SELECT COUNT(*) FROM pick_bans WHERE game_id = %s AND side = %s AND pick_ban = %s AND order_number = %s'''

In [15]:
# Player Performances
insert_player_performance = '''INSERT INTO player_performances (game_id, player_id, role, champion, kills, deaths, assists, damage_to_champions, wards_placed, wards_destroyed, control_wards_bought, vision_score, total_gold, gold_spent, creep_score, kills_at_15, deaths_at_15, assists_at_15, gold_at_15)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

# player_performance_count = '''SELECT COUNT(*) FROM player_performances WHERE game_id = %s AND player_id = %s AND role = %s AND champion = %s AND kills = %s AND deaths = %s AND assists = %s AND damage_to_champions = %s AND wards_placed = %s AND wards_destroyed = %s AND control_wards_bought = %s AND vision_score = %s AND total_gold = %s AND gold_spent = %s AND creep_score = %s AND kills_at_15 = %s AND deaths_at_15 = %s AND assists_at_15 = %s AND gold_at_15 = %s'''
player_performance_count = '''SELECT COUNT(*) FROM player_performances WHERE game_id = %s AND player_id = %s AND role = %s AND champion = %s'''

#### Script

In [16]:
csv_files

['2014_LoL_esports_match_data_from_OraclesElixir.csv',
 '2015_LoL_esports_match_data_from_OraclesElixir.csv',
 '2016_LoL_esports_match_data_from_OraclesElixir.csv',
 '2017_LoL_esports_match_data_from_OraclesElixir.csv',
 '2018_LoL_esports_match_data_from_OraclesElixir.csv',
 '2019_LoL_esports_match_data_from_OraclesElixir.csv',
 '2020_LoL_esports_match_data_from_OraclesElixir.csv',
 '2021_LoL_esports_match_data_from_OraclesElixir.csv',
 '2022_LoL_esports_match_data_from_OraclesElixir.csv',
 '2023_LoL_esports_match_data_from_OraclesElixir.csv',
 '2024_LoL_esports_match_data_from_OraclesElixir.csv',
 '2025_LoL_esports_match_data_from_OraclesElixir.csv']

##### Test

In [ ]:
for csv in csv_files[9:10]:
    df = get_dataframe(csv)
    tournaments = get_tournaments(df)
    teams = get_teams(df)
    players = get_players(df)
    
    for player in players:
        print(player)
    break

    year_teams = {}
    year_players = {}

    for team in teams:
        db_cursor.execute(team_count, (team.name, team.year))
        if db_cursor.fetchone()[0] == 0:
            db_cursor.execute(insert_team, (team.name, team.year))
            database.commit()

        print(team)

        db_cursor.execute(team_id, (team.name, team.year))
        year_teams[team.name] = db_cursor.fetchone()[0]

##### Actual

In [17]:
db_cursor = database.cursor()
# for csv in csv_files:
for csv in csv_files[11:12]:
    df = get_dataframe(csv)
    tournaments = get_tournaments(df)
    teams = get_teams(df)
    players = get_players(df)

    for tournament in tournaments:
        tournament_df = df[df['league'] == tournament.league]
        tournament_df = tournament_df[tournament_df['year'] == tournament.year]
        tournament_df = tournament_df[tournament_df['split'] == tournament.split]

        print(tournament)
        tournament.matches = get_matches(tournament_df)

    year_teams = {}
    year_players = {}

    for team in teams:
        # print(team)
        db_cursor.execute(team_count, (team.name, team.year))
        if db_cursor.fetchone()[0] == 0:
            db_cursor.execute(insert_team, (team.name, team.year))
            database.commit()

        db_cursor.execute(team_id, (team.name, team.year))
        year_teams[team.name] = db_cursor.fetchone()[0]

    for player in players:
        # print(player)
        db_cursor.execute(team_id, (player.team.name, player.team.year))
        player_team_id = db_cursor.fetchone()[0]

        # print(player.name, player_team_id)
        db_cursor.execute(player_count, (player.name, player_team_id))
        if db_cursor.fetchone()[0] == 0:
            db_cursor.execute(insert_player, (player.name, player_team_id))
            database.commit()

        db_cursor.execute(player_id, (player.name, player_team_id))
        year_players[player.name] = db_cursor.fetchone()[0]

    for tournament in tournaments:
        db_cursor.execute(tournament_count, (tournament.league, tournament.year, tournament.split))
        if db_cursor.fetchone()[0] == 0:
            db_cursor.execute(insert_tournament, (tournament.league, tournament.year, tournament.split))
            database.commit()

        db_cursor.execute(tournament_id, (tournament.league, tournament.year, tournament.split))
        current_tournament_id = db_cursor.fetchone()[0]
        
        for match in tournament.matches:
            # print(match.teamOne, match.teamTwo, match.date, match.playoffs, match.patch)
            db_cursor.execute(team_id, (match.teamOne.name, match.teamOne.year.item()))
            year_teams[match.teamOne.name] = db_cursor.fetchone()[0]
            db_cursor.execute(team_id, (match.teamTwo.name, match.teamTwo.year.item()))
            year_teams[match.teamTwo.name] = db_cursor.fetchone()[0]

            # print((current_tournament_id, match.date, year_teams[match.teamOne.name], year_teams[match.teamTwo.name], match.playoffs, match.patch))
            db_cursor.execute(match_count, (current_tournament_id, match.date, year_teams[match.teamOne.name], year_teams[match.teamTwo.name], match.playoffs, match.patch))
            if db_cursor.fetchone()[0] == 0:
                db_cursor.execute(insert_match, (current_tournament_id, match.date, year_teams[match.teamOne.name], year_teams[match.teamTwo.name], match.playoffs, match.patch))
                database.commit()

            db_cursor.execute(match_id, (current_tournament_id, match.date, year_teams[match.teamOne.name], year_teams[match.teamTwo.name], 
                                            match.playoffs, match.patch))
            current_match_id = db_cursor.fetchone()[0]

            for game in match.games:
                # print(current_match_id, game.number.item(), year_teams[game.blueTeam.name], year_teams[game.redTeam.name], game.blueWin.item(), game.duration.item())
                if math.isnan(game.number.item()):
                    continue
                db_cursor.execute(game_count, (current_match_id, game.number.item(), year_teams[game.blueTeam.name], year_teams[game.redTeam.name], game.blueWin.item(), game.duration.item()))
                if db_cursor.fetchone()[0] == 0:
                    db_cursor.execute(insert_game, (current_match_id, game.number.item(), year_teams[game.blueTeam.name], year_teams[game.redTeam.name], game.blueWin.item(), game.duration.item()))
                    database.commit()

                db_cursor.execute(game_id, (current_match_id, game.number.item(), year_teams[game.blueTeam.name], year_teams[game.redTeam.name], game.blueWin.item(), game.duration.item()))
                current_game_id = db_cursor.fetchone()[0]

                db_cursor.execute(objective_count, (current_game_id, 'Blue', game.blueObjectives.firstBlood, game.blueObjectives.firstTower, game.blueObjectives.towers, game.blueObjectives.plates, game.blueObjectives.voidGrubs, game.blueObjectives.riftHeralds, game.blueObjectives.baronNashtors, game.blueObjectives.infernalDragons, game.blueObjectives.mountainDragons, game.blueObjectives.cloudDragons, game.blueObjectives.oceanDragons, game.blueObjectives.hextechDragons, game.blueObjectives.chemtechDragons, game.blueObjectives.elderDragons, game.blueObjectives.featsOfStrength, game.blueObjectives.ruinousAtakhan, game.blueObjectives.voraciousAtakan))
                if db_cursor.fetchone()[0] == 0:
                    db_cursor.execute(insert_objective, (current_game_id, 'Blue', game.blueObjectives.firstBlood, game.blueObjectives.firstTower, game.blueObjectives.towers, game.blueObjectives.plates, game.blueObjectives.voidGrubs, game.blueObjectives.riftHeralds, game.blueObjectives.baronNashtors, game.blueObjectives.infernalDragons, game.blueObjectives.mountainDragons, game.blueObjectives.cloudDragons, game.blueObjectives.oceanDragons, game.blueObjectives.hextechDragons, game.blueObjectives.chemtechDragons, game.blueObjectives.elderDragons, game.blueObjectives.featsOfStrength, game.blueObjectives.ruinousAtakhan, game.blueObjectives.voraciousAtakan))
                    database.commit()

                db_cursor.execute(objective_count, (current_game_id, 'Red', game.redObjectives.firstBlood, game.redObjectives.firstTower, game.redObjectives.towers, game.redObjectives.plates, game.redObjectives.voidGrubs, game.redObjectives.riftHeralds, game.redObjectives.baronNashtors, game.redObjectives.infernalDragons, game.redObjectives.mountainDragons, game.redObjectives.cloudDragons, game.redObjectives.oceanDragons, game.redObjectives.hextechDragons, game.redObjectives.chemtechDragons, game.redObjectives.elderDragons, game.redObjectives.featsOfStrength, game.redObjectives.ruinousAtakhan, game.redObjectives.voraciousAtakan))
                if db_cursor.fetchone()[0] == 0:
                    db_cursor.execute(insert_objective, (current_game_id, 'Red', game.redObjectives.firstBlood, game.redObjectives.firstTower, game.redObjectives.towers, game.redObjectives.plates, game.redObjectives.voidGrubs, game.redObjectives.riftHeralds, game.redObjectives.baronNashtors, game.redObjectives.infernalDragons, game.redObjectives.mountainDragons, game.redObjectives.cloudDragons, game.redObjectives.oceanDragons, game.redObjectives.hextechDragons, game.redObjectives.chemtechDragons, game.redObjectives.elderDragons, game.redObjectives.featsOfStrength, game.redObjectives.ruinousAtakhan, game.redObjectives.voraciousAtakan))
                    database.commit()

                print(game.bluePickBan.bans)
                print(game.bluePickBan.picks)
                print(game.redPickBan.bans)
                print(game.redPickBan.picks)
                for i in range(1, 6):
                    db_cursor.execute(pick_ban_count, (current_game_id, 'Blue', 'Ban', i))
                    if db_cursor.fetchone()[0] == 0:
                        db_cursor.execute(insert_pick_ban, (current_game_id, 'Blue', 'Ban', i, game.bluePickBan.bans[i-1]))
                        database.commit()

                    db_cursor.execute(pick_ban_count, (current_game_id, 'Red', 'Ban', i))
                    if db_cursor.fetchone()[0] == 0:
                        db_cursor.execute(insert_pick_ban, (current_game_id, 'Red', 'Ban', i, game.redPickBan.bans[i-1]))
                        database.commit()

                    db_cursor.execute(pick_ban_count, (current_game_id, 'Blue', 'Pick', i))
                    if db_cursor.fetchone()[0] == 0:
                        db_cursor.execute(insert_pick_ban, (current_game_id, 'Blue', 'Pick', i, game.bluePickBan.picks[i-1]))
                        database.commit()

                    db_cursor.execute(pick_ban_count, (current_game_id, 'Red', 'Pick', i))
                    if db_cursor.fetchone()[0] == 0:
                        db_cursor.execute(insert_pick_ban, (current_game_id, 'Red', 'Pick', i, game.redPickBan.picks[i-1]))
                        database.commit()

                # db_cursor.execute(pick_ban_count, (current_game_id, 'Blue'))
                # if db_cursor.fetchone()[0] == 0:
                #     print((current_game_id, 'Blue', game.bluePickBan.ban1, game.bluePickBan.ban2, game.bluePickBan.ban3, game.bluePickBan.ban4, game.bluePickBan.ban5, game.bluePickBan.pick1, game.bluePickBan.pick2, game.bluePickBan.pick3, game.bluePickBan.pick4, game.bluePickBan.pick5))
                #     db_cursor.execute(insert_pick_ban, (current_game_id, 'Blue', game.bluePickBan.ban1, game.bluePickBan.ban2, game.bluePickBan.ban3, game.bluePickBan.ban4, game.bluePickBan.ban5, game.bluePickBan.pick1, game.bluePickBan.pick2, game.bluePickBan.pick3, game.bluePickBan.pick4, game.bluePickBan.pick5))
                #     database.commit()

                # db_cursor.execute(pick_ban_count, (current_game_id, 'Red'))
                # if db_cursor.fetchone()[0] == 0:
                #     db_cursor.execute(insert_pick_ban, (current_game_id, 'Red', game.redPickBan.ban1, game.redPickBan.ban2, game.redPickBan.ban3, game.redPickBan.ban4, game.redPickBan.ban5, game.redPickBan.pick1, game.redPickBan.pick2, game.redPickBan.pick3, game.redPickBan.pick4, game.redPickBan.pick5))
                #     database.commit()
                
                for player in game.bluePlayerPerformances:
                    # print((player.player.name, year_teams[player.player.team.name]))
                    db_cursor.execute(player_id, (player.player.name, year_teams[player.player.team.name]))
                    year_players[player.player.name] = db_cursor.fetchone()[0]

                    db_cursor.execute(player_performance_count, (current_game_id, year_players[player.player.name], player.role, player.champion))
                    if db_cursor.fetchone()[0] == 0:
                        db_cursor.execute(insert_player_performance, (current_game_id, year_players[player.player.name], player.role, player.champion, player.kills, player.deaths, player.assists, player.damageToChampions, player.wardsPlaced, player.wardsKilled, player.controlWardsBought, player.visionScore, player.totalGold, player.goldSpent, player.creepScore, player.killsAt15, player.deathsAt15, player.assistsAt15, player.goldAt15))
                        database.commit()

                for player in game.redPlayerPerformances:
                    db_cursor.execute(player_id, (player.player.name, year_teams[player.player.team.name]))
                    year_players[player.player.name] = db_cursor.fetchone()[0]

                    db_cursor.execute(player_performance_count, (current_game_id, year_players[player.player.name], player.role, player.champion))
                    if db_cursor.fetchone()[0] == 0:
                        # print(current_game_id, year_players[player.player.name], player.player.name)
                        db_cursor.execute(insert_player_performance, (current_game_id, year_players[player.player.name], player.role, player.champion, player.kills, player.deaths, player.assists, player.damageToChampions, player.wardsPlaced, player.wardsKilled, player.controlWardsBought, player.visionScore, player.totalGold, player.goldSpent, player.creepScore, player.killsAt15, player.deathsAt15, player.assistsAt15, player.goldAt15))
                        database.commit()

C:\Users\ktann\AppData\Local\Temp\ipykernel_205552\3123184168.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


LFL2 2025 - Winter
LCKC 2025 - Kickoff
LVP SL 2025 - Winter
LCK 2025 - Cup
NLC 2025 - Winter
LCP 2025 - Split 1
LEC 2025 - Winter
HLL 2025 - Winter
PRM 2025 - Winter
LFL 2025 - Flash In
LIT 2025 - Winter
TCL 2025 - Winter
HW 2025 - N/A
LJL 2025 - N/A
LTA S 2025 - Split 1
LTA N 2025 - Split 1
RL 2025 - Winter
LJL 2025 - Forge
NEXO 2025 - Split 2
CD 2025 - N/A
EBL 2025 - Winter
PCS 2025 - Split 1
ROL 2025 - Winter
LPLOL 2025 - Winter
AL 2025 - Winter
LTA 2025 - N/A
VCS 2025 - Spring
FST 2025 - N/A
PCS 2025 - N/A
EM 2025 - Winter
CD 2025 - Split 1
LRS 2025 - Split 1
LRN 2025 - Split 1
PRM 2025 - Spring
ROL 2025 - Spring
NACL 2025 - Split 1
TCL 2025 - Spring
LAS 2025 - Champ 1
LEC 2025 - Spring
LCKC 2025 - Rounds 1-2
HLL 2025 - Spring
LCK 2025 - Rounds 1-2
HC 2025 - Spring
NLC 2025 - Spring
LIT 2025 - Spring
LJL 2025 - Storm
PRMP 2025 - N/A
LTA S 2025 - Split 2
LTA N 2025 - Split 2
PCS 2025 - Split 2
HM 2025 - Spring
RL 2025 - Spring
LVP SL 2025 - Spring
LFL 2025 - Spring
AL 2025 - Spring


In [ ]:
for team in teams:
    db_cursor.execute(team_count, (team.name, team.year))
    if db_cursor.fetchone()[0] == 0:
        db_cursor.execute(insert_team, (team.name, team.year))
        database.commit()

In [ ]:
for player in players:
    db_cursor.execute(team_id, (player.team.name, player.team.year))
    player_team_id = db_cursor.fetchone()[0]

    db_cursor.execute(player_count, (player.name, player_team_id))
    if db_cursor.fetchone()[0] == 0:
        db_cursor.execute(insert_player, (player.name, player_team_id))
        database.commit()

In [50]:
db_cursor = database.cursor()

for tournament in tournaments:
    db_cursor.execute(tournament_count, (tournament.league, tournament.year, tournament.split))
    if db_cursor.fetchone()[0] == 0:
        db_cursor.execute(insert_tournament, (tournament.league, tournament.year, tournament.split))
        database.commit()

    db_cursor.execute(tournament_id, (tournament.league, tournament.year, tournament.split))
    current_tournament_id = db_cursor.fetchone()[0]

    teams = {}
    players = {}
    for match in tournament.matches:
        db_cursor.execute(team_id, (match.teamOne.name, match.teamOne.year.item()))
        teams[match.teamOne.name] = db_cursor.fetchone()[0]
        db_cursor.execute(team_id, (match.teamTwo.name, match.teamTwo.year.item()))
        teams[match.teamTwo.name] = db_cursor.fetchone()[0]

        # print((current_tournament_id, match.date, teams[match.teamOne.name], teams[match.teamTwo.name], match.playoffs, match.patch))
        db_cursor.execute(match_count, (current_tournament_id, match.date, teams[match.teamOne.name], teams[match.teamTwo.name], match.playoffs, match.patch))
        if db_cursor.fetchone()[0] == 0:
            db_cursor.execute(insert_match, (current_tournament_id, match.date, teams[match.teamOne.name], teams[match.teamTwo.name], match.playoffs, match.patch))
            database.commit()

        db_cursor.execute(match_id, (current_tournament_id, match.date, teams[match.teamOne.name], teams[match.teamTwo.name], 
                                        match.playoffs, match.patch))
        current_match_id = db_cursor.fetchone()[0]

        for game in match.games:
            db_cursor.execute(game_count, (current_match_id, game.number.item(), teams[game.blueTeam.name], teams[game.redTeam.name], game.blueWin.item(), game.duration.item()))
            if db_cursor.fetchone()[0] == 0:
                db_cursor.execute(insert_game, (current_match_id, game.number.item(), teams[game.blueTeam.name], teams[game.redTeam.name], game.blueWin.item(), game.duration.item()))
                database.commit()

            db_cursor.execute(game_id, (current_match_id, game.number.item(), teams[game.blueTeam.name], teams[game.redTeam.name], game.blueWin.item(), game.duration.item()))
            current_game_id = db_cursor.fetchone()[0]

            db_cursor.execute(objective_count, (current_game_id, 'Blue', game.blueObjectives.firstBlood, game.blueObjectives.firstTower, game.blueObjectives.towers, game.blueObjectives.plates, game.blueObjectives.voidGrubs, game.blueObjectives.riftHeralds, game.blueObjectives.baronNashtors, game.blueObjectives.infernalDragons, game.blueObjectives.mountainDragons, game.blueObjectives.cloudDragons, game.blueObjectives.oceanDragons, game.blueObjectives.hextechDragons, game.blueObjectives.chemtechDragons, game.blueObjectives.elderDragons, game.blueObjectives.featsOfStrength, game.blueObjectives.ruinousAtakhan, game.blueObjectives.voraciousAtakan))
            if db_cursor.fetchone()[0] == 0:
                db_cursor.execute(insert_objective, (current_game_id, 'Blue', game.blueObjectives.firstBlood, game.blueObjectives.firstTower, game.blueObjectives.towers, game.blueObjectives.plates, game.blueObjectives.voidGrubs, game.blueObjectives.riftHeralds, game.blueObjectives.baronNashtors, game.blueObjectives.infernalDragons, game.blueObjectives.mountainDragons, game.blueObjectives.cloudDragons, game.blueObjectives.oceanDragons, game.blueObjectives.hextechDragons, game.blueObjectives.chemtechDragons, game.blueObjectives.elderDragons, game.blueObjectives.featsOfStrength, game.blueObjectives.ruinousAtakhan, game.blueObjectives.voraciousAtakan))
                database.commit()

            db_cursor.execute(objective_count, (current_game_id, 'Red', game.redObjectives.firstBlood, game.redObjectives.firstTower, game.redObjectives.towers, game.redObjectives.plates, game.redObjectives.voidGrubs, game.redObjectives.riftHeralds, game.redObjectives.baronNashtors, game.redObjectives.infernalDragons, game.redObjectives.mountainDragons, game.redObjectives.cloudDragons, game.redObjectives.oceanDragons, game.redObjectives.hextechDragons, game.redObjectives.chemtechDragons, game.redObjectives.elderDragons, game.redObjectives.featsOfStrength, game.redObjectives.ruinousAtakhan, game.redObjectives.voraciousAtakan))
            if db_cursor.fetchone()[0] == 0:
                db_cursor.execute(insert_objective, (current_game_id, 'Red', game.redObjectives.firstBlood, game.redObjectives.firstTower, game.redObjectives.towers, game.redObjectives.plates, game.redObjectives.voidGrubs, game.redObjectives.riftHeralds, game.redObjectives.baronNashtors, game.redObjectives.infernalDragons, game.redObjectives.mountainDragons, game.redObjectives.cloudDragons, game.redObjectives.oceanDragons, game.redObjectives.hextechDragons, game.redObjectives.chemtechDragons, game.redObjectives.elderDragons, game.redObjectives.featsOfStrength, game.redObjectives.ruinousAtakhan, game.redObjectives.voraciousAtakan))
                database.commit()

            db_cursor.execute(pick_ban_count, (current_game_id, 'Blue', game.bluePickBan.ban1, game.bluePickBan.ban2, game.bluePickBan.ban3, game.bluePickBan.ban4, game.bluePickBan.ban5, game.bluePickBan.pick1, game.bluePickBan.pick2, game.bluePickBan.pick3, game.bluePickBan.pick4, game.bluePickBan.pick5))
            if db_cursor.fetchone()[0] == 0:
                db_cursor.execute(insert_pick_ban, (current_game_id, 'Blue', game.bluePickBan.ban1, game.bluePickBan.ban2, game.bluePickBan.ban3, game.bluePickBan.ban4, game.bluePickBan.ban5, game.bluePickBan.pick1, game.bluePickBan.pick2, game.bluePickBan.pick3, game.bluePickBan.pick4, game.bluePickBan.pick5))
                database.commit()

            db_cursor.execute(pick_ban_count, (current_game_id, 'Red', game.redPickBan.ban1, game.redPickBan.ban2, game.redPickBan.ban3, game.redPickBan.ban4, game.redPickBan.ban5, game.redPickBan.pick1, game.redPickBan.pick2, game.redPickBan.pick3, game.redPickBan.pick4, game.redPickBan.pick5))
            if db_cursor.fetchone()[0] == 0:
                db_cursor.execute(insert_pick_ban, (current_game_id, 'Red', game.redPickBan.ban1, game.redPickBan.ban2, game.redPickBan.ban3, game.redPickBan.ban4, game.redPickBan.ban5, game.redPickBan.pick1, game.redPickBan.pick2, game.redPickBan.pick3, game.redPickBan.pick4, game.redPickBan.pick5))
                database.commit()
            
            for player in game.bluePlayerPerformances:
                # print((player.player.name, teams[player.player.team.name]))
                db_cursor.execute(player_id, (player.player.name, teams[player.player.team.name]))
                players[player.player.name] = db_cursor.fetchone()[0]

                db_cursor.execute(player_performance_count, (current_game_id, players[player.player.name], player.role, player.champion, player.kills, player.deaths, player.assists, player.damageToChampions, player.wardsPlaced, player.wardsKilled, player.controlWardsBought, player.visionScore, player.totalGold, player.goldSpent, player.creepScore, player.killsAt15, player.deathsAt15, player.assistsAt15, player.goldAt15))
                if db_cursor.fetchone()[0] == 0:
                    db_cursor.execute(insert_player_performance, (current_game_id, players[player.player.name], player.role, player.champion, player.kills, player.deaths, player.assists, player.damageToChampions, player.wardsPlaced, player.wardsKilled, player.controlWardsBought, player.visionScore, player.totalGold, player.goldSpent, player.creepScore, player.killsAt15, player.deathsAt15, player.assistsAt15, player.goldAt15))
                    database.commit()

            for player in game.redPlayerPerformances:
                db_cursor.execute(player_id, (player.player.name, teams[player.player.team.name]))
                players[player.player.name] = db_cursor.fetchone()[0]

                db_cursor.execute(player_performance_count, (current_game_id, players[player.player.name], player.role, player.champion, player.kills, player.deaths, player.assists, player.damageToChampions, player.wardsPlaced, player.wardsKilled, player.controlWardsBought, player.visionScore, player.totalGold, player.goldSpent, player.creepScore, player.killsAt15, player.deathsAt15, player.assistsAt15, player.goldAt15))
                if db_cursor.fetchone()[0] == 0:
                    # print(current_game_id, players[player.player.name], player.player.name)
                    db_cursor.execute(insert_player_performance, (current_game_id, players[player.player.name], player.role, player.champion, player.kills, player.deaths, player.assists, player.damageToChampions, player.wardsPlaced, player.wardsKilled, player.controlWardsBought, player.visionScore, player.totalGold, player.goldSpent, player.creepScore, player.killsAt15, player.deathsAt15, player.assistsAt15, player.goldAt15))
                    database.commit()

TypeError: 'NoneType' object is not subscriptable

#### Champion

In [12]:
insert_champion = """INSERT INTO champion (champion_name, champion_true_name, champion_id, champion_title) Values (%s, %s, %s, %s)"""
get_champion = """SELECT count(*) FROM champion WHERE champion_name = %s"""

In [14]:
import requests

db_cursor = database.cursor()

# URL of the JSON data
url = 'https://ddragon.leagueoflegends.com/cdn/15.8.1/data/en_US/champion.json'

# Fetch the JSON data from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data
    # print(data)             # Print the data
    print(data['data'])
else:
    print(f"Failed to retrieve data: {response.status_code}")

for champion in data['data']:
    print(champion)
    print(data['data'][champion]['key'])
    print(data['data'][champion]['title'])

    db_cursor.execute(get_champion, (champion, ))
    count = db_cursor.fetchone()[0]

    if count == 0:
        db_cursor.execute(insert_champion, (champion, champion, data['data'][champion]['key'], data['data'][champion]['title']))
        database.commit()

{'Aatrox': {'version': '15.8.1', 'id': 'Aatrox', 'key': '266', 'name': 'Aatrox', 'title': 'the Darkin Blade', 'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...', 'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4}, 'image': {'full': 'Aatrox.png', 'sprite': 'champion0.png', 'group': 'champion', 'x': 0, 'y': 0, 'w': 48, 'h': 48}, 'tags': ['Fighter'], 'partype': 'Blood Well', 'stats': {'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpperlevel': 0, 'movespeed': 345, 'armor': 38, 'armorperlevel': 4.8, 'spellblock': 32, 'spellblockperlevel': 2.05, 'attackrange': 175, 'hpregen': 3, 'hpregenperlevel': 0.5, 'mpregen': 0, 'mpregenperlevel': 0, 'crit': 0, 'critperlevel': 0, 'attackdamage': 60, 'attackdamageperlevel': 5, 'attackspeedperlevel': 2.5, 'attackspeed': 0.651}}, 'Ahri':